In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import numpy.ma as ma
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Do NOT run, will take forever, creates a winrate file by team and maps
# team_map_wr = {}
# for team in set_of_teams:
#     team_wr = {}
#     for csgo_map in csgo_results['_map'].unique():

#         games_by_team = csgo_results[csgo_results['team_1']==team]
#         map_games = games_by_team[games_by_team['_map']==csgo_map]
#         try:
#             map_wins = map_games.groupby('map_winner').count().at[1,'date']
#         except:
#             map_wins = 0

#         try:
#             map_losses = map_games.groupby('map_winner').count().at[2,'date']
#         except:
#             map_losses = 0

#         games_by_team = csgo_results[csgo_results['team_2']==team]
#         map_games = games_by_team[games_by_team['_map']==csgo_map]
#         try:
#             map_wins += map_games.groupby('map_winner').count().at[2,'date']
#         except:
#             pass

#         try:
#             map_losses += map_games.groupby('map_winner').count().at[1,'date']
#         except:
#             pass
        
#         if map_losses == 0:
#             if map_wins == 0:
#                 map_wr = 0.0
#             else:
#                 map_wr = 1.0
#         else:
#             map_wr = map_wins/(map_wins+map_losses)

#         team_wr[csgo_map] = map_wr
#     team_map_wr[team] = team_wr
# team_map_wr['team_name']=set_of_teams
# team_map_wr = pd.DataFrame.from_dict(team_map_wr)
# team_map_wr.to_csv('team_map_wr.csv', index = False)

In [ ]:
import pandas as pd

#Cleaning data from original dataset
csgo_maps = pd.read_csv("/kaggle/input/csgo-professional-matches/picks.csv")
csgo_maps = csgo_maps[csgo_maps['t2_removed_2'] != '0.0']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Luminosity']
csgo_maps = csgo_maps[csgo_maps['left_over'] != 'Alma']
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)
csgo_maps = csgo_maps.drop(csgo_maps.index[755])
csgo_maps = csgo_maps[csgo_maps['date']>='2019-07-15'].reset_index(drop=True)
csgo_maps = csgo_maps[csgo_maps['best_of']=='3']

team_wrs = pd.read_csv("/kaggle/input/winrates/team_map_wr-2.csv")

#Getting team win rates per map
def team_map(team, csgo_map):
    try:
        x, = team_wrs[team_wrs['team_name'] == team][csgo_map]
    except:
        x = 0.0
    return x

csgo_maps['t1_map1_wr']=csgo_maps.apply(lambda x: team_map(x['team_1'],x['t1_picked_1']), axis=1)
csgo_maps['t1_map2_wr']=csgo_maps.apply(lambda x: team_map(x['team_1'],x['t2_picked_1']), axis=1)
csgo_maps['t1_map3_wr']=csgo_maps.apply(lambda x: team_map(x['team_1'],x['left_over']), axis=1)
csgo_maps['t2_map1_wr']=csgo_maps.apply(lambda x: team_map(x['team_2'],x['t1_picked_1']), axis=1)
csgo_maps['t2_map2_wr']=csgo_maps.apply(lambda x: team_map(x['team_2'],x['t2_picked_1']), axis=1)
csgo_maps['t2_map3_wr']=csgo_maps.apply(lambda x: team_map(x['team_2'],x['left_over']), axis=1)
csgo_maps.to_csv('csgo_maps.csv', index = False)
csgo_maps

In [ ]:
import pandas as pd

#Changing map names to numbers in dataset
csgo_maps = pd.read_csv("/kaggle/working/csgo_maps.csv")
csgo_results = pd.read_csv("/kaggle/input/csgo-professional-matches/results.csv")
maps_to_num = dict(zip(csgo_results['_map'].unique(), range(len(csgo_results['_map'].unique()))))

#Defines variables to use with algorithm
X_maps = csgo_maps[['team_1','team_2', 't1_picked_1', 't2_picked_1', 'left_over', 'match_id','t1_map1_wr','t1_map2_wr','t1_map3_wr','t2_map1_wr','t2_map2_wr','t2_map3_wr']]
X_results = csgo_results[['match_id', 'rank_1', 'rank_2', 'match_winner']]
csgo_df = X_maps.merge(X_results, on='match_id')
y = csgo_df[['match_winner']]
X = csgo_df[['team_1','team_2','t1_picked_1', 't2_picked_1', 'left_over', 't1_map1_wr','t1_map2_wr','t1_map3_wr','t2_map1_wr','t2_map2_wr','t2_map3_wr','rank_1', 'rank_2']]

#Create a list of all teams and number them
set_of_teams = set(list(X['team_1']) + list(X['team_2']))
team_to_num = dict(zip(set_of_teams, range(len(set_of_teams))))
#Replace team's name with their number
X['team_1'] = X['team_1'].replace(team_to_num)
X['team_2'] = X['team_2'].replace(team_to_num)

#Replace maps with numbers
X['t1_picked_1'] = X['t1_picked_1'].replace(maps_to_num)
X['t2_picked_1'] = X['t2_picked_1'].replace(maps_to_num)
X['left_over'] = X['left_over'].replace(maps_to_num)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#Prediction Algorithm
clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
score = clf.score(X_test, y_test)
score

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

#Test cases taken from BLAST Premier
new_matches = {'team_1': ['BIG', 'Vitality', 'Complexity', 'Astralis', 'FURIA', 'Astralis','G2','BIG','NiP','NiP','Chaos'],
               'team_2': ['FaZe', 'BIG', 'FaZe', 'MIBR', 'G2','G2','MIBR','Complexity','Natus Vincere','Evil Geniuses','Liquid'],
               'rank_1': [6,3,14,2,4,2,13,6,10,10,20],
               'rank_2': [8,6,8,234,13,13,234,14,5,7,12],
               't1_removed_1': ['Train','Mirage','Inferno','Mirage','Dust2','Mirage','Overpass','Train','Dust2','Dust2','Vertigo'],
               't2_removed_1': ['Vertigo','Vertigo','Vertigo','Overpass','Overpass','Overpass','Inferno','Inferno','Vertigo','Vertigo','Train'],
               't1_picked_1': ['Nuke','Nuke','Nuke','Nuke','Nuke','Nuke','Dust2','Dust2','Overpass','Overpass','Nuke'],
               't2_picked_1': ['Mirage','Dust2','Mirage','Vertigo','Mirage','Dust2','Nuke','Vertigo','Nuke','Inferno','Dust2'],
               't1_removed_2': ['Overpass','Train','Train','Dust2','Inferno','Vertigo','Vertigo','Overpass','Train','Train','Inferno'],
               't2_removed_2': ['Inferno','Overpass','Overpass','Train','Train','Train','Mirage','Mirage','Inferno','Mirage','Mirage'],
               'left_over': ['Dust2','Inferno','Dust2','Inferno','Vertigo','Inferno','Train','Nuke','Mirage','Nuke','Overpass']}

new_matches = pd.DataFrame(data=new_matches)

team_wrs = pd.read_csv("/kaggle/input/winrates/team_map_wr-2.csv")

def team_map(team, csgo_map):
    try:
        x, = team_wrs[team_wrs['team_name'] == team][csgo_map]
    except:
        x = 0.0
    return x

new_matches['t1_map1_wr']=new_matches.apply(lambda x: team_map(x['team_1'],x['t1_picked_1']), axis=1)
new_matches['t1_map2_wr']=new_matches.apply(lambda x: team_map(x['team_1'],x['t2_picked_1']), axis=1)
new_matches['t1_map3_wr']=new_matches.apply(lambda x: team_map(x['team_1'],x['left_over']), axis=1)
new_matches['t2_map1_wr']=new_matches.apply(lambda x: team_map(x['team_2'],x['t1_picked_1']), axis=1)
new_matches['t2_map2_wr']=new_matches.apply(lambda x: team_map(x['team_2'],x['t2_picked_1']), axis=1)
new_matches['t2_map3_wr']=new_matches.apply(lambda x: team_map(x['team_2'],x['left_over']), axis=1)

X_test = new_matches[['team_1','team_2','t1_picked_1', 't2_picked_1', 'left_over', 't1_map1_wr','t1_map2_wr','t1_map3_wr','t2_map1_wr','t2_map2_wr','t2_map3_wr','rank_1', 'rank_2']]

#Replace team's name with their number
X_test['team_1'] = X_test['team_1'].replace(team_to_num)
X_test['team_2'] = X_test['team_2'].replace(team_to_num)

#Replace maps with numbers
X_test['t1_picked_1'] = X_test['t1_picked_1'].replace(maps_to_num)
X_test['t2_picked_1'] = X_test['t2_picked_1'].replace(maps_to_num)
X_test['left_over'] = X_test['left_over'].replace(maps_to_num)


clf = RandomForestClassifier(max_depth=10)
clf.fit(X, y)
new_matches[['actual_winner']] = [1,1,2,1,2,1,1,1,2,1,0]
new_matches[['predicted_winner']] = clf.predict(X_test)
new_matches[['team_1','team_2','predicted_winner','actual_winner']]